# Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

## Imports y preparación dataset

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import random

In [2]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))
display(newsgroups_test.target_names)

tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}
y_train = newsgroups_train.target

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Ejemplo similitud de documentos 

In [ ]:
idx = 4811
display(newsgroups_train.data[idx])
cossim = cosine_similarity(X_train[idx], X_train)[0]
np.sort(cossim)[::-1]       #Valores de similaridad ordenados de mayor a menor
np.argsort(cossim)[::-1]    #Indices de documentos similares de mayor a menor
mostsim = np.argsort(cossim)[::-1][1:6] #Top 5
for i in mostsim:
  display(newsgroups_train.target_names[y_train[i]])

"THE WHITE HOUSE\n\n                  Office of the Press Secretary\n                   (Pittsburgh, Pennslyvania)\n______________________________________________________________\nFor Immediate Release                         April 17, 1993     \n\n             \n                  RADIO ADDRESS TO THE NATION \n                        BY THE PRESIDENT\n             \n                Pittsburgh International Airport\n                    Pittsburgh, Pennsylvania\n             \n             \n10:06 A.M. EDT\n             \n             \n             THE PRESIDENT:  Good morning.  My voice is coming to\nyou this morning through the facilities of the oldest radio\nstation in America, KDKA in Pittsburgh.  I'm visiting the city to\nmeet personally with citizens here to discuss my plans for jobs,\nhealth care and the economy.  But I wanted first to do my weekly\nbroadcast with the American people. \n             \n             I'm told this station first broadcast in 1920 when\nit reported th

'talk.politics.misc'

'talk.politics.misc'

'talk.politics.misc'

'talk.politics.misc'

'talk.politics.misc'

### Ejemplo clasificación

In [4]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)
display(f"F1 score {f1_score(y_test, y_pred, average='macro')}")

'F1 score 0.5854345727938506'

## Desarrollo 

### Punto 1

In [81]:
def get_docs_cosine_similarity(X_train, idx, n):
    topn_dict = {"doc_idx": idx}
    cossim = cosine_similarity(X_train[idx], X_train)[0]    
    topn_dict.update({"docs_sim_idx" : np.argsort(cossim)[::-1][1:n+1], "docs_sim_cosin" : np.sort(cossim)[::-1][1:n+1]})
    return topn_dict

def print_doc_info(dataset, topn_docs):
    for docs in topn_docs:
        text = dataset.data[docs['doc_idx']]
        type = dataset.target_names[int(dataset.target[docs['doc_idx']])]
        display(f"Doc referencia ID:{docs['doc_idx']}, clase: {type}")
        display(f"CONTENIDO: {text}")
        for i in range(len(docs['docs_sim_idx'])):
            text = dataset.data[docs['docs_sim_idx'][i]]
            type = dataset.target_names[int(dataset.target[docs['docs_sim_idx'][i]])]
            display(f"{i+1}) Doc similar ID: {docs['docs_sim_idx'][i]}, similitud coseno: {docs['docs_sim_cosin'][i]}, clase: {type}")
            display(f"CONTENIDO: {text}")
        display('-'*150)

In [82]:
seed = 42
random.seed(seed)
n_docs = 5
sample_docs_idx = random.sample(range(X_train.shape[0]), n_docs)

topn_docs = []
for i in sample_docs_idx:
    topn_docs.append(get_docs_cosine_similarity(X_train, i, 5))
print_doc_info(newsgroups_train, topn_docs)


'Doc referencia ID:10476, clase: rec.sport.hockey'

"CONTENIDO: This is a general question for US readers:\n\nHow extensive is the playoff coverage down there?  In Canada, it is almost\nimpossible not to watch a series on TV (ie the only two series I have not had\nan opportunity to watch this year are Wash-NYI and Chi-Stl, the latter because\nI'm in the wrong time zone!).  We (in Canada) are basically swamped with \ncoverage, and I wonder how many series/games are televised nationally or even\nlocally in the US and how much precedence they take over, say, local news if\nthe games go into double-OT.\n\nEmail me so as not to waste bandwidth, please.  My news feed is kind of slow\nanyways."

'1) Doc similar ID: 5064, similitud coseno: 0.22503645427838304, clase: rec.sport.hockey'

"CONTENIDO: \nI only have one comment on this:  You call this a *classic* playoff year\nand yet you don't include a Chicago-Detroit series.  C'mon, I'm a Boston\nfan and I even realize that Chicago-Detroit games are THE most exciting\ngames to watch."

'2) Doc similar ID: 9623, similitud coseno: 0.21743229780919432, clase: talk.politics.mideast'

...

'4) Doc similar ID: 7308, similitud coseno: 0.13177395037929182, clase: rec.sport.hockey'

"CONTENIDO: Detroit is a very disciplined team.  There's a lot of Europeans\nin Detroit which would make the game fast, so Toronto would have\nto slow the game down, which means drawing penalties, as a last\nresort anyway.  Toronto will be a good team as soon as they get\nmore good players.  Toronto is just an average team, Detroit isn't\nBallard screwed Toronto when he was owner.  Everyone knows that.\nand it's going to take time for Toronto to become a real force.\nI expect Gilmour to be burnt out next year.  He can't pull the\nwhole team forever."

'5) Doc similar ID: 10792, similitud coseno: 0.13073210977865626, clase: rec.sport.baseball'

'CONTENIDO: \n\nThe tribe will be in town from April 16 to the 19th.\nThere are ALWAYS tickets available! (Though they are playing Toronto,\nand many Toronto fans make the trip to Cleveland as it is easier to\nget tickets in Cleveland than in Toronto.  Either way, I seriously\ndoubt they will sell out until the end of the season.)\n'

'------------------------------------------------------------------------------------------------------------------------------------------------------'

Los resultados obtenidos indican que la similitud coseno aplicada a vectores TF-IDF es una herramienta útil para identificar documentos con contenido relacionado al texto de referencia, tanto desde el punto de vista semántico como categórico siempre y cuando exista una alta correlación. 

Cuando los valores de similitud son elevados, es común que los documentos recuperados pertenezcan a la misma clase y compartan términos específicos del tema. No obstante, en situaciones donde la similitud medida es baja, los documentos obtenidos tienden a no guardar una conexión semántica clara ni coincidir en categoría, lo que sugiere que la vectorización no logra reflejar adecuadamente el contenido textual.

### Punto 2

In [89]:
def cls_test (X_train_tfidf, X_test_tfidf):
    clf = MultinomialNB()
    clf.fit(X_train_tfidf, y_train)
    y_pred =  clf.predict(X_test_tfidf)
    display(f"F1 score para MultinomialNB {f1_score(y_test, y_pred, average='macro')}")

    clf = ComplementNB()
    clf.fit(X_train_tfidf, y_train)
    y_pred =  clf.predict(X_test_tfidf)
    display(f"F1 score para ComplementNB {f1_score(y_test, y_pred, average='macro')}")

In [104]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

tfidfvect = TfidfVectorizer()
X_train_tfidf = tfidfvect.fit_transform(newsgroups_train.data)
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}
y_train = newsgroups_train.target
X_test_tfidf = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

display("TF-IDF default")
cls_test (X_train_tfidf, X_test_tfidf)

tfidfvect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.9, min_df=2)
X_train_tfidf = tfidfvect.fit_transform(newsgroups_train.data)
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}
y_train = newsgroups_train.target
X_test_tfidf = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

display("TF-IDF con parámetros")
cls_test (X_train_tfidf, X_test_tfidf)

'TF-IDF default'

'F1 score para MultinomialNB 0.5854345727938506'

'F1 score para ComplementNB 0.692953349950875'

'TF-IDF con parámetros'

'F1 score para MultinomialNB 0.6503941723196313'

'F1 score para ComplementNB 0.7037173109758078'

Se compararon los modelos MultinomialNB y ComplementNB utilizando representaciones TF-IDF, tanto con la configuración por defecto como con parámetros ajustados (eliminación de stop words en inglés, uso de n-gramas y filtros de frecuencia). 

Esta diferencia se debe a que ComplementNB fue diseñado como una mejora del MultinomialNB para manejar mejor clases desbalanceadas o términos poco representativos, lo que lo vuelve más robusto en configuraciones básicas. Sin embargo, al aplicar una vectorización más cuidada, MultinomialNB mejora notablemente (de 0.585 a 0.650), reduciendo la brecha de desempeño con ComplementNB, que también mejora ligeramente a 0.704.

Estos resultados muestran que ComplementNB es más tolerante a configuraciones simples, mientras que MultinomialNB se beneficia significativamente de una selección adecuada de características.

### Punto 3

In [ ]:
def get_words_cosine_similarity(X_train_T, tfidfvect, word, n):
    topn_dict = {"word": word}
    try:
        word_idx = tfidfvect.vocabulary_[word]
    except:
        print(f"{word} no in vocabulary")
    topn_dict.update({"word_idx": word_idx})
    cossim = cosine_similarity(X_train_T[word_idx], X_train_T)[0]
    topn_dict.update({"word_sim_idx" : np.argsort(cossim)[::-1][1:n+1], "word_sim_cosin" : np.sort(cossim)[::-1][1:n+1]})
    return topn_dict

def print_word_info(idx2word, topn_words):
    for words in topn_words:
        display(f"Palabra referencia {words['word']}, ID:{words['word_idx']}")
        for i in range(len(words['word_sim_idx'])):
            display(f"{i+1}) Palabra similar {idx2word[words['word_sim_idx'][i]]}, ID: {words['word_sim_idx'][i]}, similitud coseno: {words['word_sim_cosin'][i]}")
        display('-'*150)

In [135]:
tfidfvect = TfidfVectorizer(stop_words='english', max_df=0.9, min_df=2) #No voy a utilizar ngrams para que no me busque por pares de palabras
X_train_tfidf = tfidfvect.fit_transform(newsgroups_train.data)
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}
y_train = newsgroups_train.target

X_train_T = X_train_tfidf.transpose()
words2test = ['motorcycle', 'cpu', 'football', 'atheist', 'toronto']
topn_words = []
for word in words2test:
    topn_words.append(get_words_cosine_similarity(X_train_T, tfidfvect, word, 5))
print_word_info(idx2word, topn_words)

'Palabra referencia motorcycle, ID:23793'

'1) Palabra similar wheelie, ID: 37637, similitud coseno: 0.404315501394809'

'2) Palabra similar shaft, ID: 31561, similitud coseno: 0.2692545582932047'

'3) Palabra similar learner, ID: 21091, similitud coseno: 0.23783674199976998'

'4) Palabra similar bluntly, ID: 7229, similitud coseno: 0.2302796637099688'

...

'2) Palabra similar gilmour, ID: 16314, similitud coseno: 0.2479111071502925'

'3) Palabra similar chicago, ID: 8879, similitud coseno: 0.24710485460503068'

'4) Palabra similar eecg, ID: 13301, similitud coseno: 0.19442096527109395'

'5) Palabra similar disciplined, ID: 12269, similitud coseno: 0.1834678555585414'

'------------------------------------------------------------------------------------------------------------------------------------------------------'

Transponer la matriz TF-IDF de documentos genera una matriz que representa términos en función de los documentos en los que aparecen. El significado de cada término queda definido por su distribución a lo largo del corpus, lo cual está limitado por la cantidad y la categoría de los documentos disponibles.

En los ejemplos seleccionados manualmente se observa que términos como 'motorcycle' están fuertemente asociados a 'wheelie' y 'shaft', o que 'cpu' se relaciona con 'heatsink', 'cooler' o 'tasking'. De forma similar, Toronto muestra correlación con otras ciudades como Detroit o Chicago. Sin embargo, el método no es completamente preciso, ya que algunas asociaciones carecen de sentido aparente, posiblemente porque las relaciones entre palabras se manifiestan a nivel de documento y no de coocurrencia directa.

TF-IDF este método no considera contexto, orden de palabras ni sintaxis. Para mejorar la representación semántica de las palabras, se recurre a técnicas más avanzadas como los word embeddings, que capturan relaciones contextuales de forma más robusta.